# check GPU availabilty

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Number of GPUs Available: ", len(physical_devices))
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

Number of GPUs Available:  0


In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
DRIVE_PATH = '/content/drive/MyDrive/Bachelor_Project/'
#%cd '/content/drive/MyDrive/Bachelor_Project/'
#%ls

# Load Dataset from kaggle

In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#! mkdir ~/.kaggle
#! cp kaggle.json ~/.kaggle/
#! chmod 600 ~/.kaggle/kaggle.json
#! kaggle datasets download mozillaorg/common-voice

In [ ]:
#!unzip /content/common-voice.zip

# spoken-language-identification dataset


In [ ]:
! mkdir ~/.kaggle
! cp "/content/drive/MyDrive/Bachelor_Project/kaggle.json" ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download toponowicz/spoken-language-identification

100% 14.9G/14.9G [03:30<00:00, 78.9MB/s]
100% 14.9G/14.9G [03:30<00:00, 76.2MB/s]


In [ ]:
!unzip /content/spoken-language-identification.zip
!mkdir /content/train/train-img/
!mkdir /content/test/test-img/

Streaming output truncated to the last 5000 lines.
  inflating: train/train/es_m_b53274d3ef7996a22d6ec929c1869804.fragment16.pitch5.flac  
  inflating: train/train/es_m_b53274d3ef7996a22d6ec929c1869804.fragment16.pitch6.flac  
  inflating: train/train/es_m_b53274d3ef7996a22d6ec929c1869804.fragment16.pitch7.flac  
  inflating: train/train/es_m_b53274d3ef7996a22d6ec929c1869804.fragment16.pitch8.flac  
  inflating: train/train/es_m_b53274d3ef7996a22d6ec929c1869804.fragment16.speed1.flac  
  inflating: train/train/es_m_b53274d3ef7996a22d6ec929c1869804.fragment16.speed2.flac  
  inflating: train/train/es_m_b53274d3ef7996a22d6ec929c1869804.fragment16.speed3.flac  
  inflating: train/train/es_m_b53274d3ef7996a22d6ec929c1869804.fragment16.speed4.flac  
  inflating: train/train/es_m_b53274d3ef7996a22d6ec929c1869804.fragment16.speed5.flac  
  inflating: train/train/es_m_b53274d3ef7996a22d6ec929c1869804.fragment16.speed6.flac  
  inflating: train/train/es_m_b53274d3ef7996a22d6ec929c1869804.fragme

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import librosa
import librosa.display
import gc

In [ ]:
train_data_dir=np.array(glob("/content/train/train/*"))
test_data_dir=np.array(glob("/content/test/test/*"))

In [ ]:
print(len(train_data_dir))

73080


## Create spectogram

In [ ]:
train_size= 100
test_size= 100

In [ ]:
def create_spectrogram(filename,name, is_train=True):
    plt.interactive(False)
    clip, sample_rate = librosa.load(filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    #S = librosa.feature.mfcc(y=clip, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    prefix= '/content/train/train-img/'
    if not is_train:
        prefix= '/content/test/test-img/'
    filename  = prefix + name
    plt.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()
    fig.clf()
    plt.close(fig)
    plt.close('all')
    del filename,name,clip,sample_rate,fig,ax,S

In [ ]:
#%%time
#i=0
#train_id = []
#for file in train_data_dir[i:i+train_size]:
#    name = file.split('/')[-1] + '.jpg'
#    train_id.append(name+".jpg")
#    create_spectrogram(file,name)
#
#traindf = pd.DataFrame(train_data_dir[i:i+train_size],columns = ['ID'])

CPU times: user 32.8 s, sys: 11.3 s, total: 44.1 s
Wall time: 36.9 s


In [ ]:
%%time
#for file in train_data_dir[i:i+train_size]:
#    name = file.split('/')[-1]
#    train_id.append(name+".jpg")
#    create_spectrogram(file,name)

traindf = pd.DataFrame(train_data_dir[0:train_size],columns = ['file'])
traindf['ID']= traindf['file'].apply(lambda x: x.split('/')[-1]+'.jpg')
#traindf.apply(lambda x: create_spectrogram(x.file, x.ID), axis=1)
np.vectorize(create_spectrogram)(traindf['file'], traindf['ID'])
traindf

CPU times: user 22.7 s, sys: 11.1 s, total: 33.8 s
Wall time: 22.6 s


,file,ID
0,/content/train/train/en_f_386ee651f6f1539ff562...,en_f_386ee651f6f1539ff5622c55e234e5a4.fragment...
1,/content/train/train/es_f_53b555eab2b3baada380...,es_f_53b555eab2b3baada380f7d3ede20b20.fragment...
2,/content/train/train/es_m_534a0cf6f6aea8e35667...,es_m_534a0cf6f6aea8e35667a9147d2d0f8d.fragment...
3,/content/train/train/de_m_df90bffac026f705f4a9...,de_m_df90bffac026f705f4a99871d71684f1.fragment...
4,/content/train/train/en_m_61764486dd621ce03fa4...,en_m_61764486dd621ce03fa461e92cd9e2b3.fragment...
...,...,...
95,/content/train/train/en_f_10134f409d9b7b0b95fe...,en_f_10134f409d9b7b0b95fed6e025febcad.fragment...
96,/content/train/train/de_m_df90bffac026f705f4a9...,de_m_df90bffac026f705f4a99871d71684f1.fragment...
97,/content/train/train/es_m_d5b91a4ffb1ead826b79...,es_m_d5b91a4ffb1ead826b7968ec19cbfa1c.fragment...
98,/content/train/train/es_f_bac9be13a058d1c859ed...,es_f_bac9be13a058d1c859ed5d2e3a0dc27b.fragment...


In [ ]:
traindf['lang'] = traindf['ID'].apply(lambda x: x.split('_')[0])

# save data to drive

In [ ]:
traindf.to_csv(DRIVE_PATH+'traindf.csv')

In [ ]:
#!cp traindf.csv $DRIVE_PATH
!cp /content/train/train-img/ $DRIVE_PATH -r
#!echo $DRIVE_PATH

# read data from drive

In [ ]:
traindf=pd.read_csv(DRIVE_PATH+'traindf.csv')

In [ ]:
#import shutil
#shutil.rmtree('/content/train/train-img/', ignore_errors=True)

In [ ]:
!cp $DRIVE_PATH/train-img/ /content/train/train-img/ -r

In [ ]:
!rm /content/spoken-language-identification.zip

# Parallelism

In [ ]:
!pip install swifter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 830.9/830.9 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.9 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.3.4-py3-none-any.whl size=16299 sha256=991106cbdbafc63cc368ef13a5c47bf0ed73ae6e3b0db6dfe3d8d06fe82d9987
  Stored in directory: /root/.cache/pip/wheels/6c/bd/3e/2d6afc9bc36c9975f8e4215a270bbac6580c4361ebd6bb2323
Successfully built swifter


In [ ]:
    #! rm /content/train/*.jpg

In [ ]:
gc.collect()

6674

In [ ]:
#print(train_id)
#train_lang = []
#for i in range(len(train_id)):
#    train_lang.append(train_id[i].split('_')[0])
#traindf=pd.DataFrame(train_id,columns = ['ID'])
#
#traindf['lang'] = traindf['ID'].apply(lambda x: x.split('_')[0])

In [ ]:
#traindf['lang'] = train_lang
#traindf.drop(columns='Class', inplace=True)

In [ ]:
traindf

,Unnamed: 0,file,ID,lang
0,0,/content/train/train/en_f_386ee651f6f1539ff562...,en_f_386ee651f6f1539ff5622c55e234e5a4.fragment...,en
1,1,/content/train/train/es_f_53b555eab2b3baada380...,es_f_53b555eab2b3baada380f7d3ede20b20.fragment...,es
2,2,/content/train/train/es_m_534a0cf6f6aea8e35667...,es_m_534a0cf6f6aea8e35667a9147d2d0f8d.fragment...,es
3,3,/content/train/train/de_m_df90bffac026f705f4a9...,de_m_df90bffac026f705f4a99871d71684f1.fragment...,de
4,4,/content/train/train/en_m_61764486dd621ce03fa4...,en_m_61764486dd621ce03fa461e92cd9e2b3.fragment...,en
...,...,...,...,...
95,95,/content/train/train/en_f_10134f409d9b7b0b95fe...,en_f_10134f409d9b7b0b95fed6e025febcad.fragment...,en
96,96,/content/train/train/de_m_df90bffac026f705f4a9...,de_m_df90bffac026f705f4a99871d71684f1.fragment...,de
97,97,/content/train/train/es_m_d5b91a4ffb1ead826b79...,es_m_d5b91a4ffb1ead826b7968ec19cbfa1c.fragment...,es
98,98,/content/train/train/es_f_bac9be13a058d1c859ed...,es_f_bac9be13a058d1c859ed5d2e3a0dc27b.fragment...,es


In [ ]:
!ls /content/test/test-img

In [ ]:
#!ls /content/train/train

Streaming output truncated to the last 5000 lines.
en_m_29119ce8a7ecbf970f9a0728087c8ac3.fragment18.noise4.flac
en_m_29119ce8a7ecbf970f9a0728087c8ac3.fragment18.noise5.flac
en_m_29119ce8a7ecbf970f9a0728087c8ac3.fragment18.noise6.flac
en_m_29119ce8a7ecbf970f9a0728087c8ac3.fragment18.noise7.flac
en_m_29119ce8a7ecbf970f9a0728087c8ac3.fragment18.noise8.flac
en_m_29119ce8a7ecbf970f9a0728087c8ac3.fragment18.noise9.flac
en_m_29119ce8a7ecbf970f9a0728087c8ac3.fragment18.pitch1.flac
en_m_29119ce8a7ecbf970f9a0728087c8ac3.fragment18.pitch2.flac
en_m_29119ce8a7ecbf970f9a0728087c8ac3.fragment18.pitch3.flac
en_m_29119ce8a7ecbf970f9a0728087c8ac3.fragment18.pitch4.flac
en_m_29119ce8a7ecbf970f9a0728087c8ac3.fragment18.pitch5.flac
en_m_29119ce8a7ecbf970f9a0728087c8ac3.fragment18.pitch6.flac
en_m_29119ce8a7ecbf970f9a0728087c8ac3.fragment18.pitch7.flac
en_m_29119ce8a7ecbf970f9a0728087c8ac3.fragment18.pitch8.flac
en_m_29119ce8a7ecbf970f9a0728087c8ac3.fragment18.speed1.flac
en_m_29119ce8a7ecbf970f9a0728087c8

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
#%%time
#i= train_size #start i after train_size
#test_id = []
#for file in test_data_dir[i:i+test_size]:
#    name = file.split('/')[-1]
#    test_id.append(name+'.jpg')
#    create_spectrogram(file,name, is_train=False)

testdf = pd.DataFrame(test_data_dir[:test_size],columns = ['file'])
testdf['ID']= testdf['file'].apply(lambda x: x.split('/')[-1]+'.jpg')
#testdf.apply(lambda x: create_spectrogram(x.file, x.ID), axis=1)
np.vectorize(create_spectrogram)(testdf['file'], testdf['ID'], is_train=False)
testdf['lang'] = testdf['ID'].apply(lambda x: x.split('_')[0])
testdf

,file,ID,lang
0,/content/test/test/en_f_67a0cba10d171b24039a79...,en_f_67a0cba10d171b24039a79faa1d4d603.fragment...,en
1,/content/test/test/de_f_63f5b79c76cf5a1a4bbd1c...,de_f_63f5b79c76cf5a1a4bbd1c40f54b166e.fragment...,de
2,/content/test/test/es_m_08111ce8d6a7ebc6cd2c27...,es_m_08111ce8d6a7ebc6cd2c27e62a3d98f0.fragment...,es
3,/content/test/test/de_f_63f5b79c76cf5a1a4bbd1c...,de_f_63f5b79c76cf5a1a4bbd1c40f54b166e.fragment...,de
4,/content/test/test/de_f_63f5b79c76cf5a1a4bbd1c...,de_f_63f5b79c76cf5a1a4bbd1c40f54b166e.fragment...,de
...,...,...,...
95,/content/test/test/en_f_67a0cba10d171b24039a79...,en_f_67a0cba10d171b24039a79faa1d4d603.fragment...,en
96,/content/test/test/de_m_923551d571cc437382d029...,de_m_923551d571cc437382d0294dda2dd0aa.fragment...,de
97,/content/test/test/es_m_08111ce8d6a7ebc6cd2c27...,es_m_08111ce8d6a7ebc6cd2c27e62a3d98f0.fragment...,es
98,/content/test/test/es_f_50298ab71aaba8508ebeef...,es_f_50298ab71aaba8508ebeef49d853df11.fragment...,es


In [ ]:
#testdf=pd.DataFrame(test_id,columns = ['ID'])
#
#test_lang = []
#for i in range(len(test_id)):
#    test_lang.append(test_id[i].split('_')[0])
#
#testdf['lang'] = test_lang
#

In [ ]:
testdf

,file,ID,lang
0,/content/test/test/en_f_67a0cba10d171b24039a79...,en_f_67a0cba10d171b24039a79faa1d4d603.fragment...,en
1,/content/test/test/de_f_63f5b79c76cf5a1a4bbd1c...,de_f_63f5b79c76cf5a1a4bbd1c40f54b166e.fragment...,de
2,/content/test/test/es_m_08111ce8d6a7ebc6cd2c27...,es_m_08111ce8d6a7ebc6cd2c27e62a3d98f0.fragment...,es
3,/content/test/test/de_f_63f5b79c76cf5a1a4bbd1c...,de_f_63f5b79c76cf5a1a4bbd1c40f54b166e.fragment...,de
4,/content/test/test/de_f_63f5b79c76cf5a1a4bbd1c...,de_f_63f5b79c76cf5a1a4bbd1c40f54b166e.fragment...,de
...,...,...,...
95,/content/test/test/en_f_67a0cba10d171b24039a79...,en_f_67a0cba10d171b24039a79faa1d4d603.fragment...,en
96,/content/test/test/de_m_923551d571cc437382d029...,de_m_923551d571cc437382d0294dda2dd0aa.fragment...,de
97,/content/test/test/es_m_08111ce8d6a7ebc6cd2c27...,es_m_08111ce8d6a7ebc6cd2c27e62a3d98f0.fragment...,es
98,/content/test/test/es_f_50298ab71aaba8508ebeef...,es_f_50298ab71aaba8508ebeef49d853df11.fragment...,es


In [ ]:
!pip install Keras-Preprocessing
from keras_preprocessing.image import ImageDataGenerator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.8 MB/s eta 0:00:00


In [ ]:
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.2) #.flow_from_


train_generator=datagen.flow_from_dataframe(
    dataframe=traindf,
    directory="/content/train/train-img",
    x_col="ID",
    y_col="lang",
    subset="training",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(64,64))

valid_generator=datagen.flow_from_dataframe(
    dataframe=traindf,
    directory="/content/train/train-img",
    x_col="ID",
    y_col="lang",
    subset="validation",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(64,64))



Found 80 validated image filenames belonging to 3 classes.
Found 20 validated image filenames belonging to 3 classes.


In [ ]:
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import keras

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(64,64,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer=keras.optimizers.RMSprop(lr=0.0005, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        896       
                                                                 
 activation (Activation)     (None, 64, 64, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 62, 62, 64)        18496     
                                                                 
 activation_1 (Activation)   (None, 62, 62, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 31, 31, 64)        0         
                                                        

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=150
)
model.evaluate_generator(generator=valid_generator, steps=STEP_SIZE_VALID
)

<ipython-input-40-f86d2eeb155e>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


Epoch 1/150
2/2 [==============================] - 4s 902ms/step - loss: 1.1243 - accuracy: 0.3750
Epoch 2/150
2/2 [==============================] - 2s 659ms/step - loss: 1.0985 - accuracy: 0.3333
Epoch 3/150
2/2 [==============================] - 3s 2s/step - loss: 1.0858 - accuracy: 0.3333
Epoch 4/150
2/2 [==============================] - 2s 1s/step - loss: 1.2371 - accuracy: 0.3750
Epoch 5/150
2/2 [==============================] - 2s 550ms/step - loss: 1.0915 - accuracy: 0.3542
Epoch 6/150
2/2 [==============================] - 2s 980ms/step - loss: 1.0920 - accuracy: 0.2969
Epoch 7/150
2/2 [==============================] - 2s 532ms/step - loss: 1.0849 - accuracy: 0.3958
Epoch 8/150
2/2 [==============================] - 3s 1s/step - loss: 1.0868 - accuracy: 0.3542
Epoch 9/150
2/2 [==============================] - 2s 563ms/step - loss: 1.0995 - accuracy: 0.3750
Epoch 10/150
2/2 [==============================] - 2s 986ms/step - loss: 1.0830 - accuracy: 0.3750
Epoch 11/150
2/2 [

<ipython-input-40-f86d2eeb155e>:9: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model.evaluate_generator(generator=valid_generator, steps=STEP_SIZE_VALID


[]

In [ ]:
model.save('/content/model/Model_CNN1000.h5')
gc.collect()

0

In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
    dataframe=testdf,
    directory="/content/test/test-img/",
    x_col="ID",
    y_col='lang',
    batch_size=32,
    seed=42,
    shuffle=False,
    class_mode='categorical',
    target_size=(64,64))
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

Found 100 validated image filenames belonging to 3 classes.


In [ ]:
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)
predicted_class_indices=np.argmax(pred,axis=1)

<ipython-input-61-f904d4e6328a>:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred=model.predict_generator(test_generator,


3/3 [==============================] - 1s 275ms/step


In [ ]:
predicted_class_indices.shape

(96,)

In [ ]:
from PIL import Image
import numpy as np
from skimage import transform
def load(filename):
    np_image = Image.open(filename)
    np_image = np.array(np_image).astype('float32')/255
    np_image = transform.resize(np_image, (64, 64, 3))
    np_image = np.expand_dims(np_image, axis=0)
    return np_image

image = load('/content/test/test-img/de_f_0809fd0642232f8c85b0b3d545dc2b5a.fragment18.pitch2.flac.jpg')
model.predict(image)

1/1 [==============================] - 0s 137ms/step


array([[0.00112203, 0.04741188, 0.951466  ]], dtype=float32)

In [ ]:
train_generator.class_indices

{'de': 0, 'en': 1, 'es': 2}

In [ ]:
prediction = model.predict(x=test_generator)

4/4 [==============================] - 1s 171ms/step


In [ ]:
print("accuracy on test:", np.sum(test_generator.classes == np.argmax(prediction, axis=-1)) / len(prediction) * 100)

accuracy on test: 43.0
